In [1]:
import ast
import unittest
from pathlib import Path
from typing import Union

import numpy as np
import pandas as pd
import polars as pl
import simplejson
import torch
from transformers import MBart50Tokenizer, MBartForConditionalGeneration

from simpletransformers.config import model_args as simpleconifg
from simpletransformers.seq2seq import Seq2SeqModel
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(
    data_dir: Union[Path, str],
    filename: str,
    embedding_field="embedding",
    load_embedding=True,
    ext=".json",
    parse_meta: bool = False,
    lazy: bool = False,
    sep: str = ",",
    encoding: str = "utf-8-sig",
    as_record: bool = False,
    rename_columns: dict = None,
    engine: str = "pandas",
    **kwargs,
):
    assert engine in ("pandas", "polars")
    if engine == "polars":
        import polars as pd
    else:
        import pandas as pd
    data_dir = Path(data_dir)
    db_filename = filename
    db_filepath = data_dir / (db_filename + ext)

    if ext in (".csv", ".tsv", ".xlsx", ".pickle", ".gz"):
        columns_needed = list(rename_columns.keys()) if rename_columns else None
        if ext == ".xlsx":
            df = pd.read_excel(db_filepath, engine="openpyxl") if engine == "pandas" else pd.read_excel(db_filepath)
        elif ext in (".tsv", ".csv"):
            if engine == "pandas":
                df = pd.read_csv(
                    db_filepath, encoding=encoding, usecols=columns_needed, skipinitialspace=True, sep=sep, **kwargs
                )
            else:
                df = pd.read_csv(db_filepath, encoding=encoding, sep=sep)
                df = df[columns_needed] if columns_needed else df
        elif ext in (".pickle"):
            df = pd.read_pickle(db_filepath, **kwargs)
        else:
            df = pd.read_csv(db_filepath, header=0, error_bad_lines=False, **kwargs)
        if rename_columns is not None:
            df = df.rename(rename_columns) if rename_columns else df
        if as_record:
            yield df.to_dict(orient="records")
        else:
            yield df
        raise StopIteration()
    with open(str(db_filepath), "r", encoding=encoding) as j_ptr:
        if lazy:
            for jline in j_ptr:
                yield simplejson.loads(jline)
        else:
            docs = simplejson.load(j_ptr)

    if lazy:
        raise StopIteration()

    if parse_meta:
        for d in docs:
            d["meta"] = ast.literal_eval(d["meta"])

    if embedding_field is not None:
        if load_embedding:
            index_filename = filename + "_index" + ".npy"
            index_filepath = data_dir / index_filename
            embeddings = np.load(str(index_filepath))
            for iDoc, iEmb in zip(docs, embeddings):
                iDoc[embedding_field] = iEmb
        else:
            for iDoc in docs:
                iDoc[embedding_field] = np.nan

    yield docs


In [3]:
def formify(x):
    return str(x).startswith("abstractive")


def unify(x):
    query, context = x[0], x[1]
    print(query.shape)
    print(context.shape)
    return query + " " + context


def answerify(x):
    return str(x)[len("answer:") :].strip()

In [4]:
def count_matches(labels, preds):
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [5]:
data_dir = Path.home() / "IDataset" / "dlabel"
filename = Path("ru451.csv")

model_name = "facebook/mbart-large-50"

In [6]:
model_args = {
    "model_name": f"{model_name}",
    "model_type": "mbart50",
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,  # DECODER target length as max_generated sequence since (`len(decoder_input_ids) == len(labels)`)
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "save_best_model": True,
    "max_length": 256,  # ENCODER input length. (`len(input_ids)`)
    "src_lang": "ru_RU",
    "tgt_lang": "ru_RU",
}

In [7]:
df = next(get_data(data_dir=data_dir, filename=filename.stem, ext=filename.suffix, engine="polars"))

In [8]:
df = (
            df.with_columns([pl.col("format").apply(formify).alias("mask")])
            .filter(pl.col("mask"))
            .drop("mask")
            .with_columns([pl.map(["query", "context"], unify).alias("text")])
            .with_columns([pl.col("answer").apply(answerify).alias("target")])
            .drop(["query", "context", "answer", "format", "label"])
        )

(99,)
(99,)


In [9]:
text, target = [str(x) for x in list(df.select("text").to_arrow()["text"])], [str(x) for x in df.select("target").to_arrow()["target"]]

In [10]:
train_data = [[x, y] for x, y in zip(text, target)]
train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])

In [11]:
train_df.head()

,input_text,target_text
0,query: Как Кларисса Маклеллан догадалась о про...,При первой встрече Монтэга и Клариссы Маклелла...
1,"query: Почему при встрече с Монтэгом, Кларисса...","По советом своего дяди, Кларисса ответила Монт..."
2,"query: Какое прекрасное чувство ощутил Гай, ко...",В это мгновение Гай почувствовал невероятную т...
3,"query: По какой причине, Монтэг сказал Кларисс...","Во время их первой встречи, Кларисса задает мн..."
4,query: Почему у Клариссы было очень много мысл...,Кларисса не проводила свое время как все остал...


In [12]:
model = Seq2SeqModel(
            encoder_decoder_type="mbart50", encoder_decoder_name=model_name, use_cuda=torch.cuda.is_available(), args=model_args
        )

In [13]:
model.train_model(train_df, eval_data=train_df, matches=count_matches) # This is just one epoch

(7,
 {'global_step': [7],
  'eval_loss': [3.402956792286464],
  'train_loss': [3.587454319000244],
  'matches': [0]})

In [14]:
response = model.eval_model(train_df, matches=count_matches) # You can pass any function accepting preds | labels

Generating outputs:   0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
response

{'eval_loss': 3.402956792286464, 'matches': 0}

In [16]:
query = "query: Почему у Клариссы было очень много мыслей в голове, касающиеся правильности настоящей жизни в книге «451 градусов по Фаренгейту»?"
context = "context: — Вы слишком много думаете, — заметил Монтэг, испытывая неловкость.  — Я редко смотрю телевизионные передачи, и не бываю на автомобильных гонках, и не хожу в парки развлечений. Вот у меня и остается время для всяких сумасбродных мыслей. Вы видели на шоссе за городом рекламные щиты? Сейчас они длиной в двести футов. А знаете ли вы, что когда-то они были длиной всего в двадцать футов? Но теперь автомобили несутся по дорогам с такой скоростью, что рекламы пришлось удлинить, а то их никто и прочитать бы не смог."

In [17]:
model.predict([query + context])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['query: Почему у Клариссы было очень много мыслей в голове, касающиеся правильности настоящей жизни в книге «451 градусов по Фаренгейту»?context: — Вы слишком много думаете, — заметил Монтэг, испытывая неловкость. — Я редко смотрю телевизионные передачи, и не бываю на автомобильных гонках, и не хожу в парки развлечений. Вот у меня и остается время для всяких сумасбродных мыслей. Вы видели на шоссе за городом рекламные щиты? Сейчас']

In [18]:
# Later on you can always load the best one using...

In [20]:
model = Seq2SeqModel(encoder_decoder_type="mbart50", encoder_decoder_name="outputs/best_model", use_cuda=False)

In [28]:
assert model.model.training is False

In [29]:
model.predict([query + context])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['query: Почему у Клариссы было очень много мыслей в голове, касающиеся правильности настоящей жизни в книге «451 градусов по Фаренгейту»?context: — Вы слишком много думаете, — заметил Монтэг, испытывая неловкость. — Я редко смотрю телевизионные передачи, и не бываю на автомобильных гонках, и не хожу в парки развлечений. Вот у меня и остается время для всяких сумасбродных мыслей. Вы видели на шоссе за городом рекламные щиты? Сейчас']